# Dimensionality Reduction

Many machine learning problems contain thousands or millions of features making them incredibly time consuming to train and difficult tp find good solutions for. These problems are afflicted with the _curse of dimensionality_. Many problems can reduce the number of features to form a tractable problem. Although this will speed up training, it can make the model perform slightly worse and increase pipeline complexity.

This reduction not only benefits training speed but also data visualization. Putting features on a two or three dimensional plot makes it possible to plot high-dimensional data and detect patterns.

This section will cover what happens in high-dimensional space and how to reduce dimensionality through projection and Manifold Learning as well as other techniques like PCA, Kernal PCA, and LLE.

## The Curse of Dimensionality

Random points in a 1,000,000 dimensional hypercube are on average 408.25 units apart. Compare this to a unit square's average distance of .52 and it's easy to understand how a large number of dimensions can lead to unreliable predictions. In practice high-dimensional data is often sparse leading to large distances between instance. In theory we could use enough training instances to mitigate this issue, but even a 100 (far fewer than MNIST's 784) dimension problem would require more instances than atoms in the universe in order for instances to be within .1 of each other.

## Main Approaches for Dimensionality Reduction

Two main approaches to reducing dimensionality in machine learning are through _projection_ and _manifold learning_.

### Projection

Most training instances are not uniformly distributed between dimensions, so we can _project_ the data onto a lower dimensional subspace while preserving the data's variance. Imagine our make_moons dataset but rotated on a plane in 3-dimensional space. But sometimes data twists and turns like the _Swiss Roll_ dataset, and simple projection will lead to squashed layers and difficulty drawing boundaries between instances. What we really want to do is unroll the data into the subspace.

### Manifold Learning

The Swiss roll is an example of a 2D manifold, or a hyperplane warped in higher dimensional space. More generally, a $d$-dimensional manifold is part of $n$-dimensional space (where $d < n$) that resembles a $d$-dimensional hyperplane.

Many dimensional reduction algorithms work by modeling the _manifold_ on which the training instance lie instead of the entire $n$-dimensional space. This is called _Manifold Learning_. It relies on the _manifold assumption_, or _manifold hypothesis_ that basically states that most real-world high-dimensional data usually lies close to a lower-dimensional hyperplane.

Another implicit assumption held here is that the task will become simpler after the dimensional reduction. This is not always true. Imagine a swiss roll composed of two classes bisected along the plane containing the spiral; unraveling this would create class distributions that would not contain the entire set of either class (imagine a neapolitan ice cream bar with four sections where no adjacent section is the same flavor). Reducing the dimensionality will always improve training time but will not always result in a better model.

## PCA

_Principal Component Analysis_ (PCA) is the most popular dimensionality reduction algorithm. It identifies the closest hyperplane to the dataset and then projects the data onto it.

### Preserving the Variance

Before choosing a hyperplane, the best hyperplane must be identified by choosing the one which preserves the most variance. It is most likely to lose the least amount of data in the projection. Another way to justify this choice is that it will also minimize the mean squared distance between the data points and the projection.

### Principal Components

PCA identifies the axis that accounts for the largest amount of variance. It will also find a second, third, fourth, etc. orthagonal axis for each dimension in the dataset. The unit vector that defines the i<sup>th</sup> axes is called i<sup>th</sup> _principal component_.

_Note: the direction of the principal components is not stable. Altering the training set slightly can result in a component on the same axis but facing the opposite direction._

There is a standard matrix factorization technique to identify these axes called _Singular Value Decomposition_ (SVD) that will decompose the training set matrix $X$ into the dot product of three matrices: $U \cdot \Sigma \cdot V^{T}$. Here, $V^{T}$ contains all the principal components we are looking for, see page 213 for what this matrix looks like.

The following code uses NumPy's svd() function to obtain thes principal components of a training set then extracts two PCs:

In [2]:
from sklearn.datasets import make_swiss_roll
import numpy as np

X, y = make_swiss_roll()

X_centered = X - X.mean(axis=0)
U, s, V = np.linalg.svd(X_centered)
c1 = V.T[:, 0]
c2 = V.T[:, 1]

_Note: PCA assumes the dataset is centered around the origin. Although scikit-learn's PCA classes do this automatically, make sure to do this when using own implementation._

### Projecting Down to d Dimensions

Projecting a dataset down to $d$ dimensions is a simple dot product of our training set matrix $X$ with $W_d$, a matrix defining the first $d$ principal components (the first $d$ columns of $V^T$).

$$X_{d-proj} = X \cdot W_d$$

The following python code projects the training set onto the plane defined by two principal components:

In [3]:
W2 = V.T[:, :2]
X2D = X_centered.dot(W2)

### Using Scikit-Learn

Scikit-Learn's PCA class uses SVD like above while automatically centering the data:

In [4]:
from sklearn.decomposition import PCA

pca = PCA(n_components = 2)
X2D = pca.fit_transform(X)

From here, the principal components can be accessed through the components\_ variable. This is a horizontal vector so the the first principal component can be accessed with pca.components\_.T[:, 0]

### Explained Variance Ratio

The _explained variance ratio_ indicates the proportion of the dataset's variance that comes lies along the axis of each principal component. Let's look at this value from the above data:

In [5]:
print(pca.explained_variance_ratio_)

[ 0.41340363  0.33514246]


This tells us that 41% of the variance lies along the first axis and 33% lies along the second. The remainder lies along the third and carries less importance than these two.

### Choosing the Right Number of Dimensions

Instead of arbitrarily choosing a number of dimensions, it is preferable to select the number of dimensions that add up to a sufficiently large portion of the variance (e.g. 95%). In the case of performing data visualization, then 2 or sometimes 3 dimensions is preferred.

The following code computes the PCA withoute reducing dimensionality, then computes the minimum number of dimensions to preserve 95% of the training set's variance:

In [6]:
pca = PCA()
pca.fit(X)
cumsum = np.cumsum(pca.explained_variance_ratio_)
d = np.argmax(cumsum >= 0.95) + 1

From here the n_components parameter could be set with the value of d. Alternatively, setting n_components to a float from 0.0 to 1.0 would accomplish the same goal by indicating the variance ratio we wish to preserve:

In [7]:
pca = PCA(n_components=0.95)
X_reduced = pca.fit_transform(X)

Another option is to plot the explained variance as a function of the number of dimensions. There is usually an "elbow" where the explained variance stops growing fast. This helps visualize where it stops being beneficial to include more dimensions.

### PCA for Compression

Using PCA on the MNIST dataset results in a 150 features being used instead of all 784 while preservinf 95% of the variance. This is less than 20% of the original size and can help the training of a classifier immensely.

It is also possible to decompress this reduced dataset up to 784 dimensions by applying an inverse transformation of the PCA projection. The reconstructed data will be missing some information where the 5% was originally dropped. The mean squared error between the original data and the reconstructed data is called the _reconstruction error_. After decompression, the digits suffer a slight loss of clarity but are still mostly intact.

In [8]:
from sklearn.datasets import fetch_mldata
mnist = fetch_mldata('MNIST original')

X_mnist, y_mnist = mnist["data"], mnist["target"]

pca = PCA(n_components = 154)
X_mnist_reduced = pca.fit_transform(X_mnist)
X_mnist_recovered = pca.inverse_transform(X_mnist_reduced)

Here is the equation for inverse transformation:

$$X_{recovered} = X_{d-proj} \cdot {W_d}^T$$

### Incremental PCA

The previous PCA implementations require the training set to be able to fit into memory. Luckily, _Incremental PCA_ algorithms have been developed to allow for splitting the training set into smaller subsets which can be processed one at a time. This is not only useful for large training set but also for online/on-the-fly training.

The following code splits the MNIST data set into 100 of these subsets and feeds them into Scikit-learns IncrementalPCA class. Note the use if the partial_fit() method with each subset:

In [9]:
from sklearn.decomposition import IncrementalPCA

n_batches = 100
inc_pca = IncrementalPCA(n_components=154)
for X_batch in np.array_split(X_mnist, n_batches):
    inc_pca.partial_fit(X_batch)
    
X_mnist_reduced = inc_pca.fit_transform(X_mnist)

Alternatively we can use NumPy's memmap class to manipulate a large array in a binary file as if it were in memory. This class loads only the data it needs at any given time. This pairs well with the IncrementalPCA where memory usage remains under control. We can also use the regular fit() method this way:

In [ ]:
#This code will not run without an actual file and definition for m
X_mm = np.memmap(filename, dtype_"float32", mode="readonly", shape(m, n))

batch_size = m // n_batches
inc_pca = IncrementalPCA(n_components=154, batch_size=batch_size)
inc_pca.fit(X_mm)

### Randomized PCA

A quick PCA option is _Randomized PCA_. This stochastic algorithm quickly finds an approximation of the first $d$ principal components. This can be done in $O(m \times d^2) + O(d^3)$ instead of $O(m \times n^2) + O(n^3)$. This offers signficant performance increases since $d$ is often much smaller than $n$.

In [10]:
rnd_pca = PCA(n_components=154, svd_solver="randomized")
X_reduced = rnd_pca.fit_transform(X_mnist)

## Kernel PCA

The same kernal trick discussed in chapter 5 can also be applied here in dimensional reduction called _Kernal PCA_. This method is good at preserving clusters of instances through a projection or unrolling data that lie close to a twisted manifold. The following example uses Kernal PCA with an RBF kernal:

In [12]:
from sklearn.decomposition import KernelPCA

rbf_pca = KernelPCA(n_components=2, kernel="rbf", gamma=0.04)
X_reduced = rbf_pca.fit_transform(X)

### Selecting a Kernel and Tuning Hyperparameters

Since dimensionality reduction is often a preperation step for supervised learning (the reduction itself is unsupervised), we can use a simple grid search to select a kernel and the hyperparameter values that lead to the best performance. Here's an example that first reduces dimensionality to two dimensions and then applies Logistic Regression for classification before using grid search to select the best values for accuracy:

In [14]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.datasets import make_moons

X, y = make_moons()

clf = Pipeline([
        ("kpca", KernelPCA(n_components=2)),
        ("log_reg", LogisticRegression())
    ])

param_grid =[{
        "kpca__gamma": np.linspace(0.03, 0.05, 10),
        "kpca__kernel": ["rbf", "sigmoid"]
    }]
grid_search = GridSearchCV(clf, param_grid, cv=3)
grid_search.fit(X, y)

GridSearchCV(cv=3, error_score='raise',
       estimator=Pipeline(steps=[('kpca', KernelPCA(alpha=1.0, coef0=1, copy_X=True, degree=3, eigen_solver='auto',
     fit_inverse_transform=False, gamma=None, kernel='linear',
     kernel_params=None, max_iter=None, n_components=2, n_jobs=1,
     random_state=None, remove_zero_eig=False, tol=0)), ('log_reg', LogisticRegre...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid=[{'kpca__gamma': array([ 0.03   ,  0.03222,  0.03444,  0.03667,  0.03889,  0.04111,
        0.04333,  0.04556,  0.04778,  0.05   ]), 'kpca__kernel': ['rbf', 'sigmoid']}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

Here are the optimal hyperparameters and kernal for this set:

In [15]:
print(grid_search.best_params_)

{'kpca__gamma': 0.041111111111111112, 'kpca__kernel': 'rbf'}


Another entirely unsupervised option is ti use grid search to find the hyperparameters and kernel which create the lowest reconstruction error although this is not as easy with linear PCA. Thanks to the kernel trick, a swiss roll dataset projected from three down to two dimensions through kPCA with an RBF kernel is mathematically equivalent to mapping the training set onto an infinite-dimensional feature space using the _feature map_ $\varphi$, then projecting the training set down to two dimensions with Linear PCA. If we were to invert the Linear PCA step, the reconstructed point would lie in the infinite-dimensional feature space, not the original space. Thus the reconstruction error cannot be computed. Fortunately, we can use the reconstruction _pre-image_ (a point in the original space near the reconstructed point in the feature space) and measure the pre-image error. This is done by measuring the squared distance between the pre-image and the original instance. We can then select the best hyperparameters and kernel to minimize this error.

One solution to performing this reconstruction is to train a supervised regression model with the projected instances as the training set and the original instances as the targets. Scikit-learn can do this automatically by setting fit_inverse_transform=True:

In [17]:
rbf_pca = KernelPCA(n_components=2, kernel="rbf", gamma=.041, fit_inverse_transform=True)
X_reduced = rbf_pca.fit_transform(X)
X_preimage = rbf_pca.inverse_transform(X_reduced)

_Note: the inverse\_transform method is undefined unless the fit\_inverse\_transform parameter is set._

We can then compute the reconstruction pre-image error:

In [19]:
from sklearn.metrics import mean_squared_error
mean_squared_error(X, X_reduced)

0.94802744503324232

We can then use grid-search with cross-validation to select the best kernel and hyperparamter values to minimize this error.

## LLE

_Locally Linear Embedding_ (LLE) is another powerful tool for _nonlinear dimensionality reduction_ (NLDR). Instead of relying on projections, it basically works by measuring how training instances linearly relate to it's closets neighbors (c.n.) and then looking for a low-dimensional representation of the training set to maintain these relationships. This makes it good at unrolling twisted manifolds when there is not too much noise present.

The following code uses Scikit-Learn's LocallyLinearEmbedding Class to unroll the swiss roll. It becomes completely unrolled and the distances between points are locally well preserved. However, distances are _not_ preserved globally: this can be seen in the way the edges of the manifold appear stretched and squeezed on opposite ends.

In [21]:
from sklearn.manifold import LocallyLinearEmbedding

X, y = make_swiss_roll()

lle = LocallyLinearEmbedding(n_components=2, n_neighbors=10)
X_reduced = lle.fit_transform(X)

This technique works first by identifying the $k$ closest traiing instance for each instance $x^{(i)}$. Then it tries to reconstruct $x^{(i)}$ as a linear function of these neighbors. It does this by finding the weights $w_{i,j}$ such that the squared distance between $x^{i}$ and $\sum_{j = 1}^{m} w_{i, j}x^{(j)}$ is as small as possible. This assumes $w_{i, j} = 0$ if $x^{(j)}$ is not one of $k$ closest neighbors to $x^{(i)}$. The first step of LLE is contrained by this optimization and it's second constraint normalizes the weights for each instance.

After this step, the weight matrix $\hat{W}$ encodes the linear relationships between instances. Then we use this to map the training instances in $d$ dimensional space while preserving the relationships. We need to minimize $\sum_{j = 1}^{m} {\hat{w}}_{i,j}  z^{(j)}$. This is very similar to the first step except here we keep the weights fixed and find the optimal position on the instance's image in the lower-dimensional space.

Scikit-Learn's implementation takes $O(m\log(m)n\log(k))$ for finding the $k$ mearest neighbors, $O(mnk^3)$ for optimizing the weights and $O(dm^2)$ for comstructing the low-dimensional representations. The $m^2$ term makes this solution somewhat inefficient on large datasets.

## Other Dimensionaloty Reduction Techniques

* _Multidimensional Scaling_ (MDS) reduces dimensionality while trying to preserve distances between instances.
* _Isomap_ creates a graph by trying to connect each instance to its neares neighbors then reduces the dimensionality while trying to preserve the _geodesic distances_ between the instances.
* _t-Distributed Stochastic Neighbor Embedding_ (t-SNE) reduces dimensionality while trying to keep similar instances close and dissimilar instaces apart. This is mostly used for visualization.
* _Linear Discriminant Analysis_ is a classification algorithm that learns the most discriminative axes between classes. These axes are then used tp define a hyperplane onto which we project the data. This algorithm keeps classes as far apart as possible and is especially beneficial to run before another classification algorithm